# Normalize taxa list

Cleanup the normalized taxa list from the eODP researchers for a given particular taxon group. Create csvs to import the taxa data into the database.   

In [1]:
import sys

import pandas as pd
import numpy as np

sys.path.append('../scripts/')
import normalize_taxa as nt

In [2]:
input_file = f'raw_data/taxa/Micropal_headers_PBDB_Taxonomy_notes_taxa_list_{nt.date}.csv'

taxon_group = nt.taxon_groups[0]
crosswalk_file = f"cleaned_data/taxa/taxa_crosswalk_{taxon_group}_{nt.date}.csv"
taxa_list_file = f"cleaned_data/taxa/taxa_list_{taxon_group}_{nt.date}.csv"

## Import normalized taxa list

In [3]:
df = pd.read_csv(input_file, skiprows = 9)
df.head(2)

,taxon_group,verbatim_name,name,"name to use (if different from ""name"")",name comment field,Comment,notes,Any taxon above genus,genus modifier,genus name,...,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,comments,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank
0,NaN,RESOLVED NAMES BY TAXONOMIC GROUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,benthic_forams,Pyrite,Pyrite,NaN,NaN,"not a taxa; Going into Macrostrat, not PBDB",Summer 2020: deal with later; should go in as ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.shape

(4754, 21)

drop rows with problematic taxa

In [5]:
df = df.drop(list(range(28)))
df.head(2)

,taxon_group,verbatim_name,name,"name to use (if different from ""name"")",name comment field,Comment,notes,Any taxon above genus,genus modifier,genus name,...,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,comments,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank
28,planktic_forams,Globigerina bulloides,Globigerina bulloides,NaN,NaN,NaN,"in PBDB, so it will be classified correctly; t...",NaN,NaN,Globigerina,...,NaN,NaN,bulloides,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,planktic_forams,Globigerina falconensis,Globigerina falconensis,NaN,NaN,NaN,"in PBDB, so it will be classified correctly; t...",NaN,NaN,Globigerina,...,NaN,NaN,falconensis,NaN,NaN,NaN,NaN,NaN,NaN,NaN


setup the columns

In [6]:
dict = {'Comment': 'initial_comments', 'notes': 'processing_notes'}
df.rename(columns=dict, inplace=True)

df['normalized_name'] = np.nan

## Filter taxon group

select taxa for one taxa group

In [7]:
filtered_taxa = df[df['taxon_group'] == taxon_group]
filtered_taxa.head(2)

,taxon_group,verbatim_name,name,"name to use (if different from ""name"")",name comment field,initial_comments,processing_notes,Any taxon above genus,genus modifier,genus name,...,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,comments,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,normalized_name
1509,nannofossils,Reticulofenestra circus,Reticulofenestra circus,NaN,NaN,NaN,"in PBDB, so it will be classified correctly; t...",NaN,NaN,Reticulofenestra,...,NaN,circus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1510,nannofossils,Amaurolithus delicatus,Amaurolithus delicatus,NaN,NaN,NaN,"in PBDB, so it will be classified correctly; t...",NaN,NaN,Amaurolithus,...,NaN,delicatus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
filtered_taxa.shape

(903, 22)

## Create crosswalk csv

In [9]:
fields = nt.taxa_rank_fields + nt.taxa_fields + nt.metadata_fields
filtered_taxa = pd.DataFrame(filtered_taxa, columns=fields)
filtered_taxa.head(2)

,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,initial_comments,processing_notes,comments
1509,NaN,NaN,Reticulofenestra,NaN,NaN,NaN,circus,NaN,NaN,NaN,NaN,nannofossils,Reticulofenestra circus,NaN,"in PBDB, so it will be classified correctly; t...",NaN
1510,NaN,NaN,Amaurolithus,NaN,NaN,NaN,delicatus,NaN,NaN,NaN,NaN,nannofossils,Amaurolithus delicatus,NaN,"in PBDB, so it will be classified correctly; t...",NaN


set normalized_name using the taxa fields

In [10]:
nt.add_normalized_name_column(filtered_taxa)
filtered_taxa.head(2)

,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,initial_comments,processing_notes,comments
1509,NaN,NaN,Reticulofenestra,NaN,NaN,NaN,circus,NaN,NaN,NaN,Reticulofenestra circus,nannofossils,Reticulofenestra circus,NaN,"in PBDB, so it will be classified correctly; t...",NaN
1510,NaN,NaN,Amaurolithus,NaN,NaN,NaN,delicatus,NaN,NaN,NaN,Amaurolithus delicatus,nannofossils,Amaurolithus delicatus,NaN,"in PBDB, so it will be classified correctly; t...",NaN


In [11]:
filtered_taxa.shape

(903, 16)

In [12]:
filtered_taxa = filtered_taxa.drop(filtered_taxa[filtered_taxa['normalized_name'] == ''].index)
filtered_taxa.shape

(903, 16)

In [13]:
filtered_taxa.drop_duplicates(keep='first', inplace=True, subset =['verbatim_name', 'normalized_name'])
filtered_taxa.head(5)

,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,initial_comments,processing_notes,comments
1509,NaN,NaN,Reticulofenestra,NaN,NaN,NaN,circus,NaN,NaN,NaN,Reticulofenestra circus,nannofossils,Reticulofenestra circus,NaN,"in PBDB, so it will be classified correctly; t...",NaN
1510,NaN,NaN,Amaurolithus,NaN,NaN,NaN,delicatus,NaN,NaN,NaN,Amaurolithus delicatus,nannofossils,Amaurolithus delicatus,NaN,"in PBDB, so it will be classified correctly; t...",NaN
1511,NaN,NaN,Amaurolithus,NaN,NaN,NaN,primus,NaN,NaN,NaN,Amaurolithus primus,nannofossils,Amaurolithus primus,NaN,"in PBDB, so it will be classified correctly; t...",NaN
1512,NaN,NaN,Amaurolithus,NaN,NaN,NaN,tricorniculatus,NaN,NaN,NaN,Amaurolithus tricorniculatus,nannofossils,Amaurolithus tricorniculatus,NaN,"in PBDB, so it will be classified correctly; t...",NaN
1513,NaN,NaN,Braarudosphaera,NaN,NaN,NaN,bigelowii,NaN,NaN,NaN,Braarudosphaera bigelowii,nannofossils,Braarudosphaera bigelowii,NaN,"in PBDB, so it will be classified correctly; t...",NaN


In [14]:
filtered_taxa.shape

(822, 16)

create crosswalk csv

In [15]:
filtered_taxa.to_csv(crosswalk_file, index=False)

## Create taxa list csv

create taxa list with unique taxa fields

In [16]:
fields = nt.taxa_rank_fields + nt.taxa_fields
taxa_df = pd.DataFrame(filtered_taxa, columns=fields)

# drop duplicate data
taxa_df = taxa_df.drop_duplicates() 

taxa_df.head()

,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group
1509,NaN,NaN,Reticulofenestra,NaN,NaN,NaN,circus,NaN,NaN,NaN,Reticulofenestra circus,nannofossils
1510,NaN,NaN,Amaurolithus,NaN,NaN,NaN,delicatus,NaN,NaN,NaN,Amaurolithus delicatus,nannofossils
1511,NaN,NaN,Amaurolithus,NaN,NaN,NaN,primus,NaN,NaN,NaN,Amaurolithus primus,nannofossils
1512,NaN,NaN,Amaurolithus,NaN,NaN,NaN,tricorniculatus,NaN,NaN,NaN,Amaurolithus tricorniculatus,nannofossils
1513,NaN,NaN,Braarudosphaera,NaN,NaN,NaN,bigelowii,NaN,NaN,NaN,Braarudosphaera bigelowii,nannofossils


In [17]:
taxa_df.shape

(746, 12)

### create taxa csv

In [18]:
taxa_df.to_csv(taxa_list_file, index=False)